# 3. Data Cleaning & Manipulation
With Pandas, we can do more than just select data that is already there. 
We can add new columns to our datasets, apply functions, iterate thorugh each row in the dataframe, and more.

This is where we move from "pandas for exploring our data" to "pandas for getting our data ready to feed into models".

In [1]:
import pandas as pd
import numpy as np

In [2]:
wine = pd.read_csv('data/wine/winemag-data_first150k.csv', index_col=0)
energy = pd.read_csv('data/energy/PJM_Load_hourly.csv', parse_dates=True, index_col=0)

### Dealing with Null Values:

In [3]:
print('Does the wine df conatin nulls?:', wine.isnull().any().any())
print('Does the energy df contain nulls?:', energy.isnull().any().any())

Does the wine df conatin nulls?: True
Does the energy df contain nulls?: False


In [4]:
wine.isnull().any().any()

True

**Question:** why did we use ".any()" twice when asking this question?

Now, we need to deal with the null values in the wine table.
How we deal with them will depend on the data type of the column in which null values are found, and how we hope to use the data.

If a wine doesn't have a region listed, we can still get a lot of information about it from things like its country. But, if a wine doesn't have a price listed, it's difficult to get much information out of it. 

So, if a wine has a null value for either of its regions, we will set it to be "unknown". But, if it has a null price, we will drop the row.

In [5]:
wine.isnull().any()

country         True
description    False
designation     True
points         False
price           True
province        True
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

In [6]:
print('Shape of wine table before is: ', wine.shape)

null_country = wine[wine['country'].isnull()]
print('Shape of null countries table is:', null_country.shape)

wine = wine[~wine.index.isin(null_country.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150930, 10)
Shape of null countries table is: (5, 10)
Shape of wine table after is:  (150925, 10)


The 5 rows where a NaN value existed for country have been dropped.

Now we'll do the same for rows with a null price.

In [7]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'].isnull()]
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (150925, 10)
Shape of null prices table is: (13695, 10)
Shape of wine table after is:  (137230, 10)


Wow! We lost a lot of rows getting rid of the null price rows.
Can you think of another way we could deal with null prices that doesn't mean losing almost 1/10 of our data? What are the pros and cons to using these other methods?

In [8]:
print('Shape of wine table before is: ', wine.shape)

null_price = wine[wine['price'] == 'unknown']
print('Shape of null prices table is:', null_price.shape)

wine = wine[~wine.index.isin(null_price.index)]
print('Shape of wine table after is: ', wine.shape)

Shape of wine table before is:  (137230, 10)
Shape of null prices table is: (0, 10)
Shape of wine table after is:  (137230, 10)


/anaconda3/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


Now, for the other columns in which there are nulls, we can use `df.fillna()` to populate the null values with a value of our choosing. 

This is where we have to make some decisions. Improperly setting variables can have unintended consequences on any analysis we plan to do later using this data.

In [9]:
wine.isnull().any()

country        False
description    False
designation     True
points         False
price          False
province       False
region_1        True
region_2        True
variety        False
winery         False
dtype: bool

The only columns where we still have nulls are regions and designations.
To avoid having to drop out more rows, we will use "fillna()".

In [10]:
wine.fillna('unknown', inplace=True)

We now have no null values in our dataset!

In [11]:
wine.isnull().any()

country        False
description    False
designation    False
points         False
price          False
province       False
region_1       False
region_2       False
variety        False
winery         False
dtype: bool

In [12]:
def wine_africa(row):
    if row.country in (['South Africa', 'Morocco']):
        x = 1
    else:
        x = 0
    return x


In [13]:
wine['from Africa'] = wine.apply(wine_africa,axis = 1)

In [14]:
wine[wine['from Africa'] ==1 ]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,from Africa
114,South Africa,Muted aromas of tart gooseberry and lime peel ...,Circumstance,86,20.0,Stellenbosch,unknown,unknown,Sauvignon Blanc,Waterkloof,1
216,Morocco,"Tones of black cherry, toffee, tobacco, tar an...",Domaine Excelcio,93,35.0,Guerrouane,unknown,unknown,Syrah-Grenache,Bernard Magrez,1
461,South Africa,"Muted, soft scents of green apple, orange peel...",unknown,86,20.0,Walker Bay,unknown,unknown,Sauvignon Blanc,Bouchard Finlayson,1
721,South Africa,"This wine opens softly, with faint aromas of l...",Schaapenberg Vineyard Reserve,86,35.0,Stellenbosch,unknown,unknown,Sauvignon Blanc,Vergelegen,1
724,South Africa,"This opens with scents of green melon rind, st...",Barrelman's Select,86,11.0,Western Cape,unknown,unknown,Sauvignon Blanc,Fat Barrel,1
...,...,...,...,...,...,...,...,...,...,...,...
150741,South Africa,This wine emphasizes some of the tougher eleme...,unknown,83,15.0,Stellenbosch,unknown,unknown,Pinotage,Jacobsdal,1
150744,South Africa,The attractive opening notes of orange blossom...,Chameleon,83,16.0,Stellenbosch,unknown,unknown,White Blend,Jardin,1
150746,South Africa,More than a touch of green tobacco leafiness s...,unknown,82,15.0,Coastal Region,unknown,unknown,Merlot,Saxenburg Estate,1
150750,South Africa,A medium to deep ruby color opens things up. T...,unknown,82,14.0,Paarl,unknown,unknown,Merlot,Backsberg,1


### Creating new columns from existing ones
In pandas, it's easy to make a new column from existing ones. 

In [15]:
wine.loc[:, 'Region'] = wine.loc[:, 'region_1'] + '_' + wine.loc[:, 'region_2']

#### Using a function + apply to make a new column:
Let's say we want to make a new column with an ID we can use to identify the records.
We can use a helper function + the apply method in pandas to apply it to all rows.
Apply looks through each row or column of a dataframe (depending on Axis set).

In [16]:
# finding the price of the 90th percentile price of the wines
expensive_wine_cutoff = wine.loc[:, 'price'].quantile(.90)
expensive_wine_cutoff

60.0

In [17]:
def add_unique_id(row, expensive_wine_cutoff):
    if row['price'] >= expensive_wine_cutoff:
        row['expensive_wine'] = 1
    else:
        row['expensive_wine'] = 0
    return row

In [18]:
wine = wine.apply(lambda row: add_unique_id(row, expensive_wine_cutoff), axis=1)

In [19]:
wine.head()

,country,description,designation,points,price,province,region_1,region_2,variety,winery,from Africa,Region,expensive_wine
0,US,This tremendous 100% varietal wine hails from ...,Martha's Vineyard,96,235.0,California,Napa Valley,Napa,Cabernet Sauvignon,Heitz,0,Napa Valley_Napa,1
1,Spain,"Ripe aromas of fig, blackberry and cassis are ...",Carodorum Selección Especial Reserva,96,110.0,Northern Spain,Toro,unknown,Tinta de Toro,Bodega Carmen Rodríguez,0,Toro_unknown,1
2,US,Mac Watson honors the memory of a wine once ma...,Special Selected Late Harvest,96,90.0,California,Knights Valley,Sonoma,Sauvignon Blanc,Macauley,0,Knights Valley_Sonoma,1
3,US,"This spent 20 months in 30% new French oak, an...",Reserve,96,65.0,Oregon,Willamette Valley,Willamette Valley,Pinot Noir,Ponzi,0,Willamette Valley_Willamette Valley,1
4,France,"This is the top wine from La Bégude, named aft...",La Brûlade,95,66.0,Provence,Bandol,unknown,Provence red blend,Domaine de la Bégude,0,Bandol_unknown,1


We can use the same approach to make a column for "great and cheap" wines, defined here as $50 or less and with a score >95.

In [20]:
def helper(row):
    if (row['price'] < 25) & (row['points'] >=95):
        x = 1
    else:
        x = 0
    return x

In [21]:
wine.price = wine.price.astype('int')

In [22]:
wine['great and cheap'] = wine.apply(helper, axis=1)

We can then use that boolean value to sort on the wines.

In [23]:
wine[wine['great and cheap']==1]

,country,description,designation,points,price,province,region_1,region_2,variety,winery,from Africa,Region,expensive_wine,great and cheap
20,US,Heitz has made this stellar rosé from the rare...,Grignolino,95,24,California,Napa Valley,Napa,Rosé,Heitz,0,Napa Valley_Napa,0,1
22079,US,"With residual sugar at about 1.35%, this balan...",unknown,95,20,Washington,Columbia Valley (WA),Columbia Valley,Riesling,Poet's Leap,0,Columbia Valley (WA)_Columbia Valley,0,1
51461,US,Spectacular is the word that comes to mind her...,Bacchus Vineyard,95,20,Washington,Columbia Valley (WA),Columbia Valley,Riesling,Januik,0,Columbia Valley (WA)_Columbia Valley,0,1
56971,US,"Superb fruit highlights this tight, sculpted S...",unknown,96,20,Washington,Columbia Valley (WA),Columbia Valley,Syrah,Rulo,0,Columbia Valley (WA)_Columbia Valley,0,1
85050,US,"Truly stunning, the Lewis Estate Riesling from...",Lewis Estate Vineyard,95,20,Washington,Columbia Valley (WA),Columbia Valley,Riesling,Dunham,0,Columbia Valley (WA)_Columbia Valley,0,1
102565,US,"With 21% residual sugar, this is an enormously...",Tears of Dew Late Harvest,95,22,California,Paso Robles,Central Coast,Moscato,EOS,0,Paso Robles_Central Coast,0,1
104504,US,"Simply astonishing for its price, this toasty,...",unknown,95,20,Washington,Walla Walla Valley (WA),Columbia Valley,Chardonnay,Rulo,0,Walla Walla Valley (WA)_Columbia Valley,0,1
109231,US,"Superb fruit highlights this tight, sculpted S...",unknown,96,20,Washington,Columbia Valley (WA),Columbia Valley,Syrah,Rulo,0,Columbia Valley (WA)_Columbia Valley,0,1
115165,US,This dessert wine has pushed itself to the fro...,Tears of Dew Late Harvest,95,22,California,Paso Robles,Central Coast,Moscato,EOS,0,Paso Robles_Central Coast,0,1
130255,US,"With 21% residual sugar, this is an enormously...",Tears of Dew Late Harvest,95,22,California,Paso Robles,Central Coast,Moscato,EOS,0,Paso Robles_Central Coast,0,1
